# LLM Agents
This is the third notebook for the LLM section of Comp 255.

Sections:
* Is RAG an agent?
* Tool calling with AgentBot
* RAG + AgentBot
* (Optional) Fine-tuning results

In [2]:
from llamabot import SimpleBot, StructuredBot, ChatBot, QueryBot, AgentBot
from llamabot import tool
import llamabot as lmb
import json
from pydantic import BaseModel
from pathlib import Path

/Users/bpben/Documents/wheaton_class/.pixi/envs/default/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Does RAG count as an Agent?
We build a RAG workflow in the last notebook.  But does this fit the definition of an Agent? Let's see what it does if we just want to have a nice conversation.

In [6]:
# setting the data back up
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

In [9]:
system_message = "You are a helpful assistant that answers questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name = "ollama_chat/qwen2.5",
    collection_name="documents",
    document_paths=doc_paths
)

100%|██████████| 2/2 [00:00<00:00, 591.87it/s]


In [10]:
response = query_completer('Hi how are you!')


Hello! I'm an AI assistant designed to answer questions and provide information. How can I help you today?

Is this an agent? It did respond reasonably, but it seems to be pretty preoccupied with the RAG use-case.  An agent should have "agency"; it should be able to decide the right action to take.  Sometimes that might involve the RAG workflow, but other times it won't.

Retrieval can be thought of as a tool that the agent can choose to use.  One way to specify that is to create a function that does the retrieving.  Then, through natural language, we can describe the tool and the agent can decide if it makes sense to use it to respond to an input.  

Let's look at tool use more generally using `AgentBot`:

In [15]:
# adapted from https://github.com/ericmjl/llamabot/blob/main/docs/tutorials/agentbot.md
# llamabot.tool decorator
@tool
def calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float:
    if tip_rate < 0 or tip_rate > 1.0:
        raise ValueError("Tip rate must be between 0 and 1.0")
    return bill_amount * (1 + tip_rate)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant with respect to restaurant bills.",
    functions=[calculate_total_with_tip, ],
    model_name="ollama_chat/qwen2.5",   
)

In [16]:
calculate_total_only_prompt = "My dinner was $2300 without tips. Calculate my total with an 18% tip."
response = bot(calculate_total_only_prompt, max_iterations=4)
print(response.content)

{
  "tool_name": "calculate_total_with_tip",
  "tool_args": [
     {
      "name": "bill_amount",
      "value": 2300.0
    },
    {
      "name": "tip_rate",
      "value": 0.18
    }
  ]
  , "use_cached_results": []
}{
  "tool_name": "agent_finish",
  "tool_args": [
    	{
      "name": "message",
      "value": 2714.0
    }
  ]
  , "use_cached_results": []
}2714.0


This will *usually* work.  The issue with small models (7 billion parameters in this case) is that they can sometimes output strange things, which will cause errors.  Sometimes the models can self-correct, but sometimes not.

In [20]:
@tool
def split_bill(total_amount: float, num_people: int) -> float:
    return total_amount / num_people

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant with respect to restaurant bills.",
    functions=[calculate_total_with_tip, split_bill],
    model_name="ollama_chat/qwen2.5",   
)

In [21]:
calculate_and_split_prompt = """My dinner was $2300 without tips. \
Calculate my total with an 18% tip. \
Then split the bill between 4 people."""
response = bot(calculate_and_split_prompt)
print(response.content)

{"tool_name": "calculate_total_with_tip", "tool_args": [{"name": "bill_amount", "value": 2300}, {"name": "tip_rate", "value": 0.18}],"use_cached_results": []}{"tool_name": "split_bill", "tool_args": [{"name": "total_amount", "value": 2714}, {"name": "num_people", "value": 4}], "use_cached_results": []}{"tool_name": "agent_finish", "tool_args": [],"use_cached_results": [{"arg_name": "bdc66afd", "hash_key": "2714.0"}, {"arg_name": "64d191e5", "hash_key": "678.5"}]}
Error calling agent_finish: Cached result 2714.0 not found
{"tool_name": "agent_finish", "tool_args": [{ "name": "message", "value": 678.5 }], "use_cached_results": []}678.5


The bot now could be said to have "agency"; it can decide whether to use a tool or just go ahead and respond normally.

In [22]:
response = bot('Hello how are you?')
print(response.content)

{
  "tool_name": "agent_finish",
  "tool_args": [
    {"name": "message", "value": "Hello! I'm good, thank you for asking."}
  ]
  , "use_cached_results": []
}Hello! I'm good, thank you for asking.


Now, let's make the RAG workflow a tool.  Then we can use it to answer our question.

In [ ]:
# a bot within a bot - you can likely replace this with the LanceDB query itself
system_message = "You are a helpful assistant that can answer questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]
query_completer = QueryBot(
    system_prompt=system_message,
    model_name="ollama_chat/qwen2.5",
    collection_name="documents",
    document_paths=doc_paths,
)

@tool
def information_retrieval(query: str) -> str:
    query_completer(query)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant and can retrieve information if needed.",
    functions=[information_retrieval],
    model_name="ollama_chat/",   
)

100%|██████████| 2/2 [00:00<00:00, 458.72it/s]


In [35]:
response = bot("Hello how are you?")
print(response.content)

{
  "tool_name": "agent_finish",
  "tool_args": [
    {"name": "message", "value": "Hello how are you?"}
  ]
  , "use_cached_results": []
}Hello how are you?


In [38]:
response = bot("What are the national languages of Papua New Guinea")
print(response.content)

{"tool_name": "information_retrieval", "tool_args": [{"name": "query", "value": "national languages of Papua New Guinea"}], "use_cached_results": []}According to the document, the national languages of Papua New Guinea are:

1. Tok Pisin
2. English
3. Hiri Motu{
  "tool_name": "agent_finish",
  "tool_args": [
    {
      "name": "message",
      "value": "The national languages of Papua New Guinea are Tok Pisin, English, and Hiri Motu."
    }
  ]
	          		,"use_cached_results": [
          {
            "arg_name": "query",
            "hash_key": "74234e98"
          }
        ]
}
Error calling agent_finish: agent_finish() got an unexpected keyword argument 'query'
{"tool_name": "information_retrieval", "tool_args": [{"name": "query", "value": "national languages of Papua New Guinea"}], "use_cached_results": []}According to the document, the national languages of Papua New Guinea are:

1. Tok Pisin
2. English
3. Hiri Motu{
  "tool_name": "agent_finish",
  "tool_args": [
    {
    

Again, we see some oddities, but we've managed to roughly get our RAG workflow to be an tool that the agent is able to make a decision to call with specific parameters.